In [12]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pickle

In [ ]:
links = []
for page_num in tqdm(range(15000)):
    response = requests.get(f"https://kuruc.info/to/1/{page_num}0/")
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = soup.find_all("div", class_="alcikkheader")
    links.extend(["https://kuruc.info" + title.find("a").get("href") for title in titles])

In [ ]:
links